Build a bi-directional recurrent neural network (LSTM) with TensorFlow.


References:
*   Long Short Term Memory, Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.



In [0]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets("/tmp/data", one_hot=True)

W0816 09:18:30.034116 139683358246784 deprecation.py:323] From <ipython-input-2-281fd4647692>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0816 09:18:30.035943 139683358246784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0816 09:18:30.038042 139683358246784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0816 09:18:30.515084 139683358246784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0816 09:18:30.520853 139683358246784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0816 09:18:30.623265 139683358246784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# training hyperparameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# network hyperparameters
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

In [0]:
# graph inputs
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [0]:
# Define weights
weights = {
    # hidden layer wieghts -> 2*n_hidden because of forward and backward cells
    'out':tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
# Define BiRNN
def BiRNN(x, weights, biases):
  # prepare data to match function requirements
  # current data input shape - batch, timesteps, n_input
  # required shape timesteps tensor lists of shape (batch, n_input)
  
  # unstack to get a list of 'timesteps' lists of tensors of shape (batch, n_input)
  x = tf.unstack(x, timesteps,1)
  
  # define lstm cells with tensorflow
  # forward direction cell
  lstm_forward_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
  
  #backward direction cell
  lstm_backward_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
  
  # Get lstm cell output
  try:
    outputs, _ ,_ = rnn.static_bidirectional_rnn(lstm_forward_cell, lstm_backward_cell,
                                                x, dtype=tf.float32)
  except Exception: # old tensorflow version returns only outputs not states
    outputs = rnn.static_bidirectional_rnn(lstm_forward_cell, lstm_backward_cell,
                                          x, dtype = tf.float32)
    
  return tf.matmul(outputs[-1], weights['out'])+biases['out']

In [8]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

W0816 09:32:10.605586 139683358246784 deprecation.py:323] From <ipython-input-6-d088e29a8c41>:11: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0816 09:32:10.607526 139683358246784 deprecation.py:323] From <ipython-input-6-d088e29a8c41>:19: static_bidirectional_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
W0816 09:32:10.609463 139683358246784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/rnn.py:1610: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which

In [0]:
# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

#evaluate model with test logits and dropout disabled
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initialize variables
init = tf.global_variables_initializer()

In [14]:
# start session
with tf.Session() as sess:
  sess.run(init)
  
  for step in range(1, training_steps+1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    
    # reshape data to get 28 sequence of 28 elements
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    
    # run optimization op (backprop)
    sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
    
    if step%display_step ==0 or step==1:
      # calculate batch loss and accuracy
      loss, acc = sess.run([loss_op, accuracy], feed_dict ={X:batch_x,
                                                           Y:batch_y})
      
      print("Step", step, "Loss", loss, "Accuracy", acc)
  
  print("Optimization finished")
  
  #calculate accuracy for 128 test mnist images
  test_len = 128
  test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
  test_label = mnist.test.labels[:test_len]
  print("Test Accuracy", sess.run(accuracy, feed_dict={X:test_data,
                                                      Y:test_label}))

Step 1 Loss 3.3522875 Accuracy 0.0625
Step 200 Loss 2.1302328 Accuracy 0.265625
Step 400 Loss 1.965358 Accuracy 0.4140625
Step 600 Loss 1.8396318 Accuracy 0.3828125
Step 800 Loss 1.7529037 Accuracy 0.3984375
Step 1000 Loss 1.5133581 Accuracy 0.5546875
Step 1200 Loss 1.4889281 Accuracy 0.515625
Step 1400 Loss 1.4035927 Accuracy 0.5390625
Step 1600 Loss 1.292522 Accuracy 0.6328125
Step 1800 Loss 1.2392797 Accuracy 0.640625
Step 2000 Loss 1.1576811 Accuracy 0.65625
Step 2200 Loss 1.1220514 Accuracy 0.6796875
Step 2400 Loss 1.1242509 Accuracy 0.6796875
Step 2600 Loss 1.1762081 Accuracy 0.59375
Step 2800 Loss 1.0060598 Accuracy 0.703125
Step 3000 Loss 1.0705297 Accuracy 0.65625
Step 3200 Loss 0.9471547 Accuracy 0.6796875
Step 3400 Loss 0.940985 Accuracy 0.6796875
Step 3600 Loss 1.013755 Accuracy 0.703125
Step 3800 Loss 0.9831662 Accuracy 0.6875
Step 4000 Loss 0.77612543 Accuracy 0.7734375
Step 4200 Loss 0.88715106 Accuracy 0.6875
Step 4400 Loss 0.84057367 Accuracy 0.7265625
Step 4600 Loss 0